# SPECTRE MODEL TEST

In [4]:

from scapy.all import *
import numpy as np
import tensorflow as tf
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow_io as tfio
from collections import defaultdict

In [5]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.32.0
tensorflow version: 2.12.0


In [ ]:
# KAFKA IMPORT

In [6]:
# Import PCAP File
pcap_data = rdpcap("/mnt/Data/SPECTRE/Dataset/Malware_Capture_Facility_Project/2015-09-10_winlinux.pcap")

## TEST 1

In [ ]:
# Preprocess PCAP File

def preprocess_data(pcap_data):
    # Preprocess the pcap_data and return it as a NumPy array or tensor
    pass

preprocessed_data = preprocess_data(pcap_data)

In [ ]:
def process_pcap(file_name):
    packets = pcap_data
    
    flows = defaultdict(lambda: defaultdict(list))
    
    for packet in packets:
        if 'IP' in packet:
            src_ip = packet['IP'].src
            dst_ip = packet['IP'].dst
            src_port = packet.sport if 'TCP' in packet or 'UDP' in packet else 0
            dst_port = packet.dport if 'TCP' in packet or 'UDP' in packet else 0
            protocol = packet['IP'].proto
            timestamp = packet.time
            flow_id = f"{src_ip}-{dst_ip}-{src_port}-{dst_port}-{protocol}"
            
            flows[flow_id]['packets'].append(packet)
            flows[flow_id]['timestamps'].append(timestamp)
    
    features = []
    
    for flow_id, flow_data in flows.items():
        src_ip, dst_ip, src_port, dst_port, protocol = flow_id.split('-')
        timestamps = flow_data['timestamps']
        packets = flow_data['packets']
        
        flow_duration = timestamps[-1] - timestamps[0]
        total_fwd_packets = len(packets)
        
        total_bytes = sum(len(packet) for packet in packets)
        min_packet_size = min(len(packet) for packet in packets)
        max_packet_size = max(len(packet) for packet in packets)
        avg_packet_size = total_bytes / total_fwd_packets

        feature_row = [flow_id, src_ip, src_port, dst_ip, dst_port, protocol,
                       datetime.fromtimestamp(timestamps[0]), flow_duration,
                       total_fwd_packets, total_bytes,
                       min_packet_size, max_packet_size, avg_packet_size]
        features.append(feature_row)
    
    return features



In [ ]:
file_name = pcap_data
features = process_pcap(file_name)

In [ ]:
converted_features = []
for feature_row in features:
    converted_row = [value.timestamp() if isinstance(value, datetime) else value for value in feature_row]
    converted_features.append(converted_row)

In [ ]:
# Convert the features list to a NumPy array
features_array = np.asarray(converted_features)

# If needed, convert specific columns to the desired data type
# For example, convert the flow_duration column to float64
features_array[:, 7] = features_array[:, 7].astype(np.float64)

In [ ]:
# convert the Decimal values to floats 
converted_features = []

for feature_row in features:
    converted_row = []
    for value in feature_row:
        if isinstance(value, Decimal):
            converted_row.append(float(value))
        else:
            converted_row.append(value)
    converted_features.append(converted_row)

input_data = np.array(converted_features)

In [ ]:
tensor = tf.convert_to_tensor(features_array, dtype=tf.float32)

---

## Test 2

In [ ]:
# 1. Read packets from a pcap file and extract desired features
def extract_features(packet):
    packets = pcap_data
    
    flows = defaultdict(lambda: defaultdict(list))
    
    for packet in packets:
        if 'IP' in packet:
            src_ip = packet['IP'].src
            dst_ip = packet['IP'].dst
            src_port = packet.sport if 'TCP' in packet or 'UDP' in packet else 0
            dst_port = packet.dport if 'TCP' in packet or 'UDP' in packet else 0
            protocol = packet['IP'].proto
            timestamp = packet.time
            flow_id = f"{src_ip}-{dst_ip}-{src_port}-{dst_port}-{protocol}"
            
            flows[flow_id]['packets'].append(packet)
            flows[flow_id]['timestamps'].append(timestamp)
    
    features_list = []
    
    for flow_id, flow_data in flows.items():
        src_ip, dst_ip, src_port, dst_port, protocol = flow_id.split('-')
        timestamps = flow_data['timestamps']
        packets = flow_data['packets']
        
        flow_duration = timestamps[-1] - timestamps[0]
        total_fwd_packets = len(packets)
        
        total_bytes = sum(len(packet) for packet in packets)
        min_packet_size = min(len(packet) for packet in packets)
        max_packet_size = max(len(packet) for packet in packets)
        avg_packet_size = total_bytes / total_fwd_packets

        feature_row = [flow_id, src_ip, src_port, dst_ip, dst_port, protocol,
                       datetime.fromtimestamp(timestamps[0]), flow_duration,
                       total_fwd_packets, total_bytes,
                       min_packet_size, max_packet_size, avg_packet_size]
        features.append(feature_row)
    
    return features_list

In [ ]:
packets = pcap_data
features_list = [extract_features(packet) for packet in packets]

In [ ]:
# 2. Convert the extracted features into a NumPy array
features_array = np.asarray(features_list)
print(features_array)

In [ ]:
# 3. Preprocess the NumPy array as needed
# For example, convert specific columns to the desired data type
features_array[:, 7] = features_array[:, 7].astype(np.float64)

# 4. Load the SavedModel and use it with the preprocessed NumPy array

In [ ]:
# Ensure the input shape and data type match the model's requirements
preprocessed_input = tf.convert_to_tensor(features_array, dtype=tf.float32)

---

## Test 3

https://github.com/ymirsky/Kitsune-py

In [12]:
import sys
sys.path.append('/home/aryn/spectre-dev/spectre-code/spectre-ann/kitsune_py/')
from kitsune_py.Kitsune import Kitsune
from kitsune_py.FeatureExtractor import pcap_to_features

ImportError: cannot import name 'pcap_to_features' from 'kitsune_py.FeatureExtractor' (/home/aryn/spectre-dev/spectre-code/spectre-ann/kitsune_py/FeatureExtractor.py)

# PREDICTION

In [ ]:
# Load TF Saved_model
model = tf.saved_model.load("/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/SavedModel")

In [ ]:
#predictions = model(features_array)
result = model(preprocessed_input)